<h2>Get the distances between each cardiac arrest and the nearest potential AED location<h2>

In [24]:
import numpy as np
import pandas as pd
#from math import radians
from sklearn.metrics.pairwise import haversine_distances
distances = np.load('../transformed_data/intervention_to_aed_distances_km.npy')
intervention_coordinates = pd.read_csv("../transformed_data/intervention_coordinates.csv")
potential_aed_coordinates = pd.read_csv("../transformed_data/potential_aed_locations.csv")
existing_aed_coordinates=pd.read_csv('../transformed_data/aed_locations_full.csv')
#Assign id to potential AEDs
#potential_aed_coordinates['id'] = range(len(potential_aed_coordinates))
#potential_aed_coordinates.to_csv('../transformed_data/potential_aed_locations.csv', index=False)

In [25]:
#Get a matrix of new distances
intervention_latlons = intervention_coordinates[['lat', 'lon']].values.tolist()
potential_aed_latlons = potential_aed_coordinates[['lat', 'lon']].values.tolist()
intervention_latlons_radians = np.radians(np.array(intervention_latlons))
potential_aed_latlons_radians = np.radians(np.array(potential_aed_latlons))
new_distances = haversine_distances(intervention_latlons_radians, potential_aed_latlons_radians) * 6371000/1000
new_distances.shape

(32975, 10000)

In [26]:
distances.shape

(32975, 13804)

<h2>For each optimal potential AED, find the intervention locations it saves, and the corresponding diatances. <h2>

In [27]:
min_distances = np.min(distances, axis=1)
min_new_distances = np.min(new_distances, axis=1)
potential_aed_index = np.argmin(new_distances, axis=1)
existing_aed_index = np.argmin(distances, axis=1)
closest_aed_type = np.where(min_new_distances < min_distances, 'potential', 'existing')

# Create a DataFrame to store results
interventions_closest_to_potential_aed = pd.DataFrame({
    'intervention_id': intervention_coordinates.index,
    'potential_aed_id': potential_aed_index,
    'distance_to_potential_aed': min_new_distances,
    'distance_to_existing_aed': min_distances,
    'intervention_lat': intervention_coordinates['lat'],
    'intervention_lon': intervention_coordinates['lon'],
    'existing_aed_id': existing_aed_index,
    'closest_aed_type': closest_aed_type
})

interventions_closest_to_potential_aed = interventions_closest_to_potential_aed[interventions_closest_to_potential_aed['closest_aed_type'] == 'potential']


# Extract coordinates of the closest existing AEDs
interventions_closest_to_potential_aed['existing_aed_lat'] = interventions_closest_to_potential_aed['existing_aed_id'].apply(
    lambda idx: existing_aed_coordinates.at[idx, 'lat']
)
interventions_closest_to_potential_aed['existing_aed_lon'] = interventions_closest_to_potential_aed['existing_aed_id'].apply(
    lambda idx: existing_aed_coordinates.at[idx, 'lon']
)

# Group interventions by closest potential AED index
grouped_interventions = interventions_closest_to_potential_aed.groupby('potential_aed_id').agg({
    'intervention_id': list,
    'intervention_lat': list,
    'intervention_lon': list,
    'distance_to_potential_aed': list,
    'distance_to_existing_aed': list,
    'existing_aed_id': list,
    'existing_aed_lat': list,
    'existing_aed_lon': list,
}).reset_index()

grouped_interventions['arrest_count'] = grouped_interventions['intervention_id'].apply(len)

# Add the potential AED coordinates to the grouped DataFrame
grouped_interventions['potential_aed_lat'] = grouped_interventions['potential_aed_id'].apply(
    lambda idx: potential_aed_coordinates.at[idx, 'lat']
)
grouped_interventions['potential_aed_lon'] = grouped_interventions['potential_aed_id'].apply(
    lambda idx: potential_aed_coordinates.at[idx, 'lon']
)


# Save the grouped interventions to a CSV file
grouped_interventions.to_csv('../transformed_data/interventions_closest_to_potential_aeds.csv', index=False)

In [28]:
#check the map and solve the problem
optimal_potential_aeds = grouped_interventions.nlargest(1, 'arrest_count')
optimal_potential_aeds = optimal_potential_aeds.drop(columns=['distance_to_potential_aed', 'distance_to_existing_aed'])
optimal_potential_aeds.to_csv('../transformed_data/check_the_map.csv', index=False)


